In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

In [37]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed(777)

In [38]:
trans = transforms.Compose([
    transforms.ToTensor()
])
train_data = torchvision.datasets.ImageFolder(root = 'C:/Users/ehrme/모두의 딥러닝 torch/train_data/',
                              transform = trans)

In [39]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers = 2)

In [40]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120, 2))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [41]:
net = CNN()
test_input = (torch.Tensor(3,3,128, 64))
test_out = net(test_input)

In [47]:
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)
loss_func = nn.CrossEntropyLoss()

total_batch = len(data_loader)

epochs = 15
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        optimizer.zero_grad()
        out = net(imgs)
        
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('Epoch: {} cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

Epoch: 1 cost = 0.4385242462158203
Epoch: 2 cost = 0.2896321415901184
Epoch: 3 cost = 0.27851322293281555
Epoch: 4 cost = 0.16645030677318573
Epoch: 5 cost = 0.1472213864326477
Epoch: 6 cost = 0.10517628490924835
Epoch: 7 cost = 0.09127199649810791
Epoch: 8 cost = 0.0789860337972641
Epoch: 9 cost = 0.09839479625225067
Epoch: 10 cost = 0.11116117238998413
Epoch: 11 cost = 0.091310515999794
Epoch: 12 cost = 0.04448499530553818
Epoch: 13 cost = 0.044117193669080734
Epoch: 14 cost = 0.06078755855560303
Epoch: 15 cost = 0.06702861934900284
Learning Finished!


In [48]:
print(net.layer1[0])
print(net.layer1[0].weight[0][0][0])

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0813,  0.0120, -0.0094, -0.0118,  0.1023], grad_fn=<SelectBackward>)


In [51]:
trans = torchvision.transforms.Compose([
    transforms.Resize((128, 64)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root = 'C:/Users/ehrme/모두의 딥러닝 torch/test_data/',
                                            transform = trans)

In [52]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [53]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        
        prediction = net(imgs)
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy', accuracy.item())

Accuracy 1.0
